# Manifold learning vs predictive filtering plots
These few plots are made by copy-pasting bits of code to evaluate analytical results and the background autocorrelation function. Combined in one notebook for conveniece. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as pj
import json
from scipy.signal import correlate

In [ ]:
# Saving and path variables
do_save_plots = True

# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# Aesthetic parameters
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

## Loss function comparisons in 1D lineplots

In [ ]:
def loss_vp_ou(tau, n_s, p):
    """
    Loss function with both P and v strategies, for a special case
    where background odors are orthogonal and iid, new odors are a
    concentration iid to the background times a vector uniformly
    sampled on the unit hypersphere.

    tau (float, np.ndarray): time constant of the exponentially decaying
        autocorrelation function (O-U process' autocorrelation) of each odor.
    n_s (float, np.ndarray): N_S, number of olfactory receptor dimensions
    p (dict): parameters,
        sigma^2: variance of odor concentrations
        K: number of background odors
    """
    expo = 1.0 - np.exp(-2.0/tau)
    loss = p["sigma^2"] * p["K"] * expo / (1.0 + n_s*expo)
    return loss


def loss_v_ou(tau, n_s, p):
    """ Loss with only predictive filtering v.
    """
    loss = p["K"] * p["sigma^2"] * (1.0 - np.exp(-2.0/tau))
    if hasattr(n_s, "shape"):
        loss = np.ones(n_s.shape) * loss  # Ensure full array shape
    return loss   


def loss_p_ou(tau, n_s, p):
    """ Loss with only manifold learning P.
    """
    loss = p["K"] * p["sigma^2"] / (n_s+ 1.0)
    if hasattr(tau, "shape"):
        loss = np.ones(tau.shape) * loss  # Ensure full array shape
    return loss 

def phase_boundary_v_p(tau_range):
    n_s = 1.0 / (np.exp(2.0/tau_range) - 1.0)
    return n_s

def plot_loss_vs_1_param(tau, n_s, p, figax=None):
    # Compute losses
    loss_lines = {}
    loss_fcts = {"vp":loss_vp_ou, "p":loss_p_ou, "v":loss_v_ou}
    norm_fact = p["K"] * p["sigma^2"]
    for strat in loss_fcts.keys():
        loss_lines[strat] = loss_fcts[strat](tau, n_s, p)

    # Plot
    if figax is None:
        fig, ax = plt.subplots()
    else:
        fig, ax = figax
    strat_names = {"vp":r"Combined, $\mathcal{L}_{v, P}$", 
        "p":r"$P$ only, $\mathcal{L}_P$", "v":r"$v$ only, $\mathcal{L}_v$"}
    strat_styles = {"vp":"-", "p":"--", "v":":"}
    strat_clrs = {"vp":"tab:purple", "p":"tab:red", "v":"tab:blue"}
    if isinstance(tau, np.ndarray):
        xrange = tau
        xlbl = r"Autocorrelation time, $\tau$ (steps)"
        #annot_lbl = r"Fixed $\frac{\sigma^2}{\sigma_x^2} N_\mathrm{S} = " 
        annot_lbl = r"Fixed $\tilde{N}_\mathrm{S} = " 
        annot_lbl += "{:d}$".format(n_s) 
    else:
        xrange = n_s
        #xlbl = r"Scaled olfactory dimension, $\frac{\sigma^2}{\sigma_x^2} N_\mathrm{S}$"
        xlbl = r"Scaled olfactory dimension, $\tilde{N}_\mathrm{S}$"
        annot_lbl = r"Fixed $\tau = {:d}$ steps".format(tau) 

    for strat in ["v", "p", "vp"]:
        ax.plot(
            xrange, loss_lines[strat] / norm_fact, 
            label=strat_names[strat], ls=strat_styles[strat],
            lw=2.0, color=strat_clrs[strat]
        )
    ax.set_title(annot_lbl, fontsize=ax.xaxis.label.get_size())
    ax.set(
        xlabel=xlbl,
        ylabel=r"Normalized minimized loss, $\mathcal{L} \, / \tilde{N}_\mathrm{S} \sigma^2$", 
        yscale="log", xscale="log"
    )
    ax.legend(frameon=False)
    return [fig, ax]

In [ ]:
loss_params = {
    "sigma^2": 0.16,  # Useless, just an overall scale in the end
    "K": 5  # number of odors, useless in the end, 
            # since general loss = k * 1-d back. loss
}

fig, axes = plt.subplots(1, 2, sharey="row")
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2, plt.rcParams["figure.figsize"][1]*1.25)
# Plot as a function of tau, fixed N_S
tau_range = np.geomspace(1.0, 200.0, 400)
n_s_fix = 50
figax = (fig, axes.flat[0])
plot_loss_vs_1_param(tau_range, n_s_fix, loss_params, figax=figax)

# Plot as a function of N_S, tau fixed. 
n_s_range = np.geomspace(1, 200, 400)
tau_fix = 50
figax = (fig, axes.flat[1])
plot_loss_vs_1_param(tau_fix, n_s_range, loss_params, figax=figax)
# Show ticks back
axes.flat[1].yaxis.set_tick_params(labelleft=True)

fig.tight_layout()
if do_save_plots:
    fig.savefig(
        pj(panels_folder, "supfig_loss_lineplots_manifold_vs_predictive.pdf"),
        transparent=True, bbox_inches="tight"
    )
plt.show()
plt.close()

## Autocorrelation function of the turbulent background
To determine a reasonable value of the autocorrelation time $\tau$. 

In [ ]:

# Estimator of autocorrelation based on Sokal 1995 and emcee. 
# Autocorrelation analysis. Code from emcee's documentation:
# https://emcee.readthedocs.io/en/stable/tutorials/autocorr/
def next_pow_two(n):
    i = 1
    while i < n:
        i = i << 1
    return i


def autocorr_func_1d(x, norm=True):
    x = np.atleast_1d(x)
    if len(x.shape) != 1:
        raise ValueError("invalid dimensions for 1D autocorrelation function")
    n = next_pow_two(len(x))
    # Compute the FFT and then (from that) the auto-correlation function
    f = np.fft.fft(x - np.mean(x), n=2 * n)
    acf = np.fft.ifft(f * np.conjugate(f))[: len(x)].real
    acf /= 4 * n

    # Optionally normalize
    if norm and acf[0] != 0.0:
        acf /= acf[0]

    return acf


# Automated windowing procedure following Sokal (1989)
def auto_window(taus, c):
    m = np.arange(len(taus)) < c * taus
    if np.any(m):
        return np.argmin(m)
    return len(taus) - 1


def autocorr_avg(y, c=5.0):
    """ y is shaped [n_walkers, n_samples] """
    # First compute the integrated autocorrelation time
    f = np.zeros(y.shape[1])
    for yy in y:  # loop over walkers
        f += autocorr_func_1d(yy)
    f /= len(y)
    # Use the automatic windowing described by Sokal, stop the
    # sum to compute tau_int at the auto_window position
    # The sume extends from -time to +time, here use symmetry
    # to rewrite t_int = 2*sum_1^{time} + 1
    taus = 2.0 * np.cumsum(f) - 1.0
    window = auto_window(taus, c)
    # This returns the autocorrelation time and the
    # integrated autocorrelation time,
    # equal to 1/2 + correlation time if the decay is exponential
    return f, taus[window]

def scipy_avg_correlate(sers):
    # sers is shaped [odor, time]
    s0 = sers - np.mean(sers, axis=1).reshape(-1, 1)
    corr_full = np.zeros(2*s0.shape[1]-1)
    nser = sers.shape[0]
    for i in range(nser):
        corr_full += correlate(s0[i], s0[i])
    # normalize
    corr_full /= (nser * sers.shape[1])
    corr_full /= np.amax(corr_full)
    corr = corr_full[corr_full.shape[0]//2:]
    return corr

In [ ]:
# Import pre-saved turbulent time series of odor concentration
conc_ser = np.load(pj(data_folder, "sample_turbulent_background.npz"))["nuser"]
# Extract concentration series and reshape as [odor, time]
conc_ser = conc_ser[:, :, 1].T

dt = 10.0 / 1000.0 # s  1.0  #steps  
dt_units = "s"

# Compute autocorrelation function, average over odors since they are iid
autocorr_fct, autocorr_tau = autocorr_avg(conc_ser)
trange_show = min(1000, autocorr_fct.shape[0])
print("autocorrelation time:", autocorr_tau*dt, dt_units)

# Compare to a scipy autocorrelation function  -- not necessary
# scipy_fct = scipy_avg_correlate(conc_ser)

# Plot the results
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1]*1.25)
ax.plot(np.arange(trange_show)*dt, autocorr_fct[:trange_show], color="k", lw=2.0)
# ax.plot(np.arange(trange_show)*dt, scipy_fct[:trange_show], label="Scipy", ls="--")
ax.axvline(autocorr_tau*dt, ls="--", color="xkcd:dark grey")
stepsize = 0.03  # s, 30 ms reaction time
#lbl = r"$\tau =" + r"{0:.1f}$ {1} $\sim$ {2} steps".format(autocorr_tau*dt, dt_units, int(autocorr_tau*dt / stepsize))
#lbl = "Autocorrelation time\n" + lbl
lbl = r"$\tau =" + "{0:.1f}$ {1}".format(autocorr_tau*dt, dt_units)
lbl = "Autocorrelation\ntime " + lbl
ax.annotate(lbl, color="xkcd:dark grey", xy=((autocorr_tau+100)*dt, 0.6), 
            rotation=90, ha="center", va="center")
ax.set(xlabel=f"Time difference ({dt_units})", ylabel=r"Autocorrelation $C(t)$")
ax.set_title("Turbulent odor statistics")
fig.tight_layout()
if do_save_plots:
    fig.savefig(
        pj(panels_folder, "supfig_autocorrelation_turbulent_statistics.pdf"), 
        transparent=True, bbox_inches="tight"
    )
plt.show()
plt.close()

In [ ]:
1.68 / 0.03